# UNSEEN-open

In this project, the aim is to build an open, reproducible, and transferable workflow for UNSEEN.
<!-- -- an increasingly popular method that exploits seasonal prediction systems to assess and anticipate climate extremes beyond the observed record. The approach uses pooled forecasts as plausible alternate realities. Instead of the 'single realization' of reality, pooled forecasts can be exploited to better assess the likelihood of infrequent events.  -->
The workflow consists of four steps, as illustrated below:

![title](../../graphs/Workflow.png)

In this project, UNSEEN-open is applied to assess two extreme events in 2020: February 2020 UK precipitation and the 2020 Siberian heatwave. 

February average precipitation was the highest on record in the UK: with what frequency of occurrence can February extreme precipitation events such as the 2020 event be expected?

The Siberian heatwave has broken the records as well. Could such an event be anticipation with UNSEEN? And to what extend can we expect changes in the frequency of occurrence and magnitude of these kind of events?

## Overview

Here we provide an overview of the steps taken to apply UNSEEN-open.

### Download

We want to download February precipitation over the UK and March-May average temperature over Siberia. We retrieve all SEAS5 seasonal forecasts that are forecasting the target months (i.e. February and MAM) and we retrieve ERA5 reanalysis for the same regions and variables for evaluation. 

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))
os.chdir(os.path.abspath('../../'))

In [2]:
import src.cdsretrieve as retrieve
import src.preprocess as preprocess

In [5]:
import numpy as np

In [6]:
retrieve.retrieve_SEAS5(variables = ['2m_temperature','2m_dewpoint_temperature'], target_months = [3,4,5], 
         area = [70, -11, 30, 120], years=np.arange(1981, 2021), folder = '../Siberia_example/SEAS5/')

2020-09-16 12:42:52,756 INFO Welcome to the CDS
2020-09-16 12:42:52,758 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2020-09-16 12:42:55,461 INFO Request is queued
2020-09-16 12:43:44,892 INFO Request is completed
2020-09-16 12:43:44,895 INFO Downloading http://136.156.133.41/cache-compute-0013/cache/data1/adaptor.mars.external-1600256610.5590868-10484-1-43f9f2fc-9209-4306-ba84-5fc2f56c8cbb.nc to ../Siberia_example/SEAS5/201702.nc (3.2M)
2020-09-16 12:43:45,155 INFO Download rate 12.3M/s  
2020-09-16 12:43:46,261 INFO Welcome to the CDS
2020-09-16 12:43:46,263 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2020-09-16 12:43:47,913 INFO Request is queued
2020-09-16 12:43:52,712 INFO Request is running
2020-09-16 12:44:01,189 INFO Request is completed
2020-09-16 12:44:01,191 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data3/adaptor.mars.external-1600

In [4]:
retrieve.retrieve_SEAS5(variables = 'total_precipitation', target_months = [2], 
         area = [60, -11, 50, 2], folder = '../UK_example/SEAS5/')

In [5]:
retrieve.retrieve_ERA5(variables = ['2m_temperature','2m_dewpoint_temperature'], target_months = [3,4,5], 
         area = [70, -11, 30, 120], folder = '../Siberia_example/ERA5/')

In [6]:
retrieve.retrieve_ERA5(variables = 'total_precipitation', target_months = [2], 
         area = [60, -11, 50, 2], folder = '../UK_example/ERA5/')

### Preprocess

In the preprocessing step, we first merge all downloaded files into one netcdf file.
Then the rest of the preprocessing depends on the definition of the extreme event. For example, for the UK case study, we want to extract the UK average precipitation while for the Siberian heatwave we will just used the defined area to spatially average over. For the MAM season, we still need to take the seasonal average, while for the UK we already have the average February precipitation. 

In [7]:
SEAS5_Siberia = preprocess.merge_SEAS5(folder = '../Siberia_example/SEAS5/', target_months = [3,4,5])
SEAS5_Siberia

Lead time: 02
1
12


,Array,Chunk
Bytes,387.52 MB,3.31 MB
Shape,"(3, 117, 51, 41, 132)","(1, 3, 51, 41, 132)"
Count,887 Tasks,117 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,387.52 MB,3.31 MB
Shape,"(3, 117, 51, 41, 132)","(1, 3, 51, 41, 132)"
Count,887 Tasks,117 Chunks
Type,float32,numpy.ndarray


In [8]:
SEAS5_Siberia.sel(latitude = 60, longitude = -10, time = '2000-03', number = 24, leadtime = 3).load()

<xarray.Dataset>
Dimensions:    (time: 1)
Coordinates:
    longitude  float32 -10.0
    number     int32 24
  * time       (time) datetime64[ns] 2000-03-01
    latitude   float32 60.0
    leadtime   int64 3
Data variables:
    t2m        (time) float32 279.34442
    d2m        (time) float32 275.4092
Attributes:
    Conventions:  CF-1.6
    history:      2020-09-08 09:33:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [10]:
SEAS5_UK = preprocess.merge_SEAS5(folder = '../UK_example/SEAS5/', target_months = [2])
SEAS5_UK

Lead time: 1
12
11
10
9


,Array,Chunk
Bytes,539.00 kB,15.40 kB
Shape,"(35, 25, 11, 14)","(1, 25, 11, 14)"
Count,105 Tasks,35 Chunks
Type,float32,numpy.ndarray


### Read more
Jump into the respective sections for more detail:

* **Download**
    * [1. Retrieve](1.Download/1.Retrieve.ipynb) 
* **Pre-process**
    * [2.1 Merge](2.Preprocess/2.1Merge.ipynb)  
    * [2.2 Mask](2.Preprocess/2.2Mask.ipynb)
    * [2.3 Upscale](2.Preprocess/2.3Upscale.ipynb)
* **Evaluate**
    * [3. Evaluate](3.Evaluate/3.Evaluate.ipynb)
* **Illustrate**
    